# Agent with tools
This notebook illustrates how to build a simple agent capable of using tools. 
We'll use crewAI to illustrate how agentics implements logical transduction

## Initialize environment

Note: Before executing this notebook on colab, copy your .env file in /content/drive/MyDrive/.env or otherwise you'll be prompted to enter your GEMINI_API_KEY all the times.

In [ ]:
! uv pip install agentics-py

import os
import sys
from getpass import getpass

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)


if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    # Mount your google drive
    from google.colab import drive
    drive.mount("/content/drive")
    load_dotenv("/content/drive/MyDrive/.env")
else:
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

Using Python 3.12.9 environment at: /Users/gliozzo/Code/agentics911/agentics/.venv
Audited 1 package in 14ms
In Colab: False


## Create a new tool
Wrap the Duck Duck Go Search API call into a CrewAI tool using the @tool decorator. Similarly you can turn any function of your choice into a tool. 

In [ ]:
from agentics import AG
from crewai.tools import tool
from ddgs import DDGS

## Define a Crew AI tool to get news for a given date using the DDGS search engine
@tool("fetch_data_async")
def web_search(query: str) -> str:
    """ Fetch web search results for the given query using DDGS."""
    return str(DDGS().text(query, max_results=10))    

## Define an Agent

Use crew AI agent directly create a document searcher agent, who is instructed to use the web_search tool defined above. Note that reasoning steps and other parameters can be customized. 


In [ ]:
from crewai import Agent
doc_agent = Agent(
    role="Doc Searcher",
    goal="Find answers to questions from the user using the available web search tool.",
    backstory="A helpful assistant for extensive web search reports.",
    tools=[web_search],
    reasoning=True,
    reasoning_steps=2,
    verbose=True,
    llm=AG.get_llm_provider()
)

2025-09-11 10:31:50.875 | DEBUG    | agentics.core.llm_connections:get_llm_provider:29 - No LLM provider specified. Using the first available provider.
2025-09-11 10:31:50.875 | DEBUG    | agentics.core.llm_connections:get_llm_provider:31 - Available LLM providers: ['watsonx', 'gemini', 'openai']. Using 'watsonx'


## Define a Task

In [4]:
from crewai import Task

doc_task = Task(
    description="""Your task is to perform an extensive web search about
    the following question {question} and return a document providing answers to 
    the questions that explore several interesting aspects, each of them supported 
    by pertinent information from web search.  """,
    expected_output="""A markdown document with introduction and different sections""",
    agent=doc_agent,
    markdown=True
)

## Define a Crew

Crews are enviroments in which tasks are assigned to different agents following different policies. In this simple example we define a crew with a single agent and single task. 

In [5]:
from crewai import Crew

crew = Crew(
    agents=[doc_agent],
    tasks=[doc_task],
    verbose=True,
)

## Kickoff the crew

Crews are ultimatelly analoguous to very sophisticated function definitions, whose inputs are json dictonaries. In order to put them in action you should kickoff them providing input parameters.

In the Agent Trace stack you can see the plan made by the agent and the different execution steps which might involve tool calls.

In [6]:
result = crew.kickoff(inputs={"question": input("Ask your question> ") })

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ebb06ad3-ee93-4177-910e-836823cef961                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🧠 Reasoning Plan ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ## Task Understanding and Plan                                                                                 │
│                                                                                                                 │
│  ### 1. Understanding of the Task                                                                               │
│  From my professional perspective as a Doc Searcher, the task involves conducting an extensive web search to    │
│  find the time in Italy when the first plane crashed into the Twin Towers. This requires understanding the      │
│  time difference between Italy and New York, the time of the crash, and potentially, the events surrounding     │
│  the 9/11 attacks.                                                                                              │
│                                                                                                                 │
│  ### 2. Key Steps to Complete the Task                                                                          │
│  To complete this task, I will follow these steps:                                                              │
│  - Identify the time of the first plane crash into the Twin Towers in New York time.                            │
│  - Determine the time difference between New York and Italy.                                                    │
│  - Calculate the time in Italy at the moment of the crash.                                                      │
│  - Use web search tools to verify the information and gather additional details about the event.                │
│                                                                                                                 │
│  ### 3. Approaching Challenges                                                                                  │
│  Challenges might include finding reliable sources for the exact time of the crash, understanding daylight      │
│  saving time (DST) adjustments if applicable, and ensuring the accuracy of the time difference between the two  │
│  locations. My expertise as a Doc Searcher will help in identifying credible sources and navigating through     │
│  complex information to find the most accurate data.                                                            │
│                                                                                                                 │
│  ### 4. Strategic Use of Available Tools                                                                        │
│  The primary tool available is `fetch_data_async`, which allows for asynchronous web searches. I will use this  │
│  tool to:                                                                                                       │
│  - Search for official reports or news articles from 2001 about the 9/11 attacks to find the exact time of the  │
│  first plane crash.                                                                                             │
│  - Look up time zone information and any DST adjustments that might have been in effect at the time of the      │
│  crash.                                                                                                         │
│  - Verify the time difference between New York (Eastern Daylight Time, EDT, if applicable) and Italy (Central   │
│  European Summer Time, CEST, if applicable) during September 2001.                                              │
│                                                                                                                 │
│  ### 5. Expected Outcome and Alignment with Goal                                                                │
│  The expected outcome is a markdown document that provi

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Doc Searcher                                                                                            │
│                                                                                                                 │
│  Task: Your task is to perform an extensive web search about                                                    │
│      the following question What time was in italy when the first plane creashed on the twin towers? and        │
│  return a document providing answers to                                                                         │
│      the questions that explore several interesting aspects, each of them supported                             │
│      by pertinent information from web search.                                                                  │
│                                                                                                                 │
│  Reasoning Plan:                                                                                                │
│  ## Task Understanding and Plan                                                                                 │
│                                                                                                                 │
│  ### 1. Understanding of the Task                                                                               │
│  From my professional perspective as a Doc Searcher, the task involves conducting an extensive web search to    │
│  find the time in Italy when the first plane crashed into the Twin Towers. This requires understanding the      │
│  time difference between Italy and New York, the time of the crash, and potentially, the events surrounding     │
│  the 9/11 attacks.                                                                                              │
│                                                                                                                 │
│  ### 2. Key Steps to Complete the Task                                                                          │
│  To complete this task, I will follow these steps:                                                              │
│  - Identify the time of the first plane crash into the Twin Towers in New York time.                            │
│  - Determine the time difference between New York and Italy.                                                    │
│  - Calculate the time in Italy at the moment of the crash.                                                      │
│  - Use web search tools to verify the information and gather additional details about the event.                │
│                                                                                                                 │
│  ### 3. Approaching Challenges                                                                                  │
│  Challenges might include finding reliable sources for the exact time of the crash, understanding daylight      │
│  saving time (DST) adjustments if applicable, and ensuring the accuracy of the time difference between the two  │
│  locations. My expertise as a Doc Searcher will help in identifying credible sources and navigating through     │
│  complex information to find the most accurate data.                                                            │
│                                                                                                                 │
│  ### 4. Strategic Use of Available Tools                                                                        │
│  The primary tool available is `fetch_data_async`, which allows for asynchronous web searches. I will use this  │
│  tool to:                                              

Output()

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  [{'title': 'Calculator.net Time Duration Calculator', 'href':                                                  │
│  'https://www.calculator.net/time-duration-calculator.html', 'body': 'Free calculator to get the number of      │
│  hours , minutes, and seconds between two times . Also, a full version to calculate the time duration between   │
│  two dates.'}, {'title': 'Time and Date Current Local Time in New York, New York, USA', 'href':                 │
│  'https://www.timeanddate.com/worldclock/usa/new-york', 'body': "Current local time in USA – New York – New     │
│  York . Get New York 's weather and area codes, time zone and DST. Explore New York 's sunrise and sunset,      │
│  moonrise and moonset."}, {'title': 'Calculator.net Hours Calculator', 'href':                                  │
│  'https://www.calculator.net/hours-calculator.html', 'body': 'This hours calculator computes the number of      │
│  hours and minutes between two times . A full version can calculate the hours between two times on different    │
│  dates.'}, {'title': 'GIGACalculator.com Time Calculator 📅 : Easily Add or Subtract Time', 'href':             │
│  'https://www.gigacalculator.com/calculators/time-calculator.php', 'body': 'Time calculator online: add time    │
│  or subtract time to and from a given date and time ( Time Adder). ➤ Add or subtract hours , minutes or         │
│  seconds to a given time using our time calculator. See what time will it be in XX hours , what time will it    │
│  be in XX minutes, or what time is 30 minutes from now.'}, {'title': 'Calculator.net Time Calculator', 'href':  │
│  'https://www.calculator.net/time-calculator.html', 'body': 'This free time calculator can add or subtract      │
│  time values in terms of number of days, hours , minutes, or seconds. Also, learn the different concepts of     │
│  time .'}, {'title': 'Time Calculator Hours from Now Calculator: Time in Hours Minutes Later', 'href':          │
│  'https://www.timecalculator.net/hours-from-now-calculator', 'body': 'Hours from now calculator helps you to    │
│  calculate the time hours and minutes later from now. What time will it be in hours later from now.'},          │
│  {'title': 'TimeAndDate Time Difference between New York, New York, U...                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Doc Searcher                                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The time in Italy when the first plane crashed into the Twin Towers was 2:46 p.m. CEST.                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: bbf1f3f4-9a1d-4f3b-adfc-774d80ce457f                                                                     │
│  Agent: Doc Searcher                                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: ebb06ad3-ee93-4177-910e-836823cef961                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: The time in Italy when the first plane crashed into the Twin Towers was 2:46 p.m. CEST.          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Display the results as markdown

In [ ]:
from IPython.display import Markdown, display

display(Markdown(result.raw))

<IPython.core.display.Markdown object>